# North Macedonian 2024 Election Fact Checking Pipeline
---
This notebook is part of the North Macedonian 2024 Election Misinformation Assessment project. It contains code to analyze and visualize misinformation related to the elections in North Macedonia.

##### Libraries

In [2]:
from dotenv import load_dotenv
import requests
import json
from urllib.parse import quote_plus
import tqdm
import json
import os
import csv
import pandas as pd

##### Constants

In [4]:
load_dotenv()
llama3_url = "http://localhost:11434/api/generate"
google_api = os.getenv("GOOGLE-API-KEY")
cx = os.getenv("CX")

dataset = "Dataset/VMRO-DPMNE_ad_creative_bodies.csv"
party = "VMRO-DPMNE"

## Data Collection

In [7]:
pandas_df = pd.read_csv(dataset)
print(pandas_df.head())

      ad_archive_id                                 ad_creative_bodies
0   427480589881211  Para disa ditëve në emisionin “Politiko” e kam...
1  1636678057070325  Промените се во твои раце!\nМакедонија повторн...
2   252100357930225                        За многу години Цветници!!!
3  1117244869521215  ✒️Shkruan: @[100005900348509:2048:Enis Shaqiri...
4  1595858924569626  ИЗГУБЕНА Е ДОВЕРБАТА ВО ПОЛИТИЧАРИТЕ БИДЕЈЌИ Н...


In [3]:
text = """Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren dhe nuk duhet të shërbejnë për t’i bërë karshillëk askujt. 
Propozimi i kreut të VMRO-DPMNE Hristijan Mickoski që të obligon fëmijët çdo të hënë të këndojnë himnin shtetëror pasqyron mendësinë nacionaliste të VMRO-DPMNE për të ushqyer fëmijët me nacionalizëm dhe me këtë të motivohet politika e tyre drejt izolim shoqëror dhe shtetëror. 
Kam një propozim më kreativ se çka duhet të dëgjojnë fëmijët në këtë shtet dhe atë jo çdo të hënë por çdo ditë:
- të dëgjojnë se si VMRO-DPMNE shet nacionalizëm të rrejshëm ndërsa bën argatin e agjendave anti-evropiane!
- se si VMRO-DPMNE nxit urrejtje dhe ndasi!
-se si VMRO-DPMNE po bllokon të ardhmen evropiane të gjeneratave të reja!
-se si VMRO-DPMNE po izolon shtetin!
- se si VMRO-DPMNE ushtron pushtetin kundër kombit shqiptar në komunat që udhëheqin kuadrot e saj!
Me qasje të këtij lloji, shteti rrezikon shumë prandaj është i nevojshëm një bllok më i madh me orientim të qartë për të ardhmen evropiane të vendit."""

## Data Preprocessing

### Extract Claims

In [17]:

def extract_claims(text):
    promptQuestion= """For the bellow text, extract the claims and give to me in the format of a json. The claims should in English Language. You should only return the json. Do not add any other text. The json should have only 1 key `Claims` and the value should be a list of claims."""
    
    extract_claim_payload = {
    "model": "llama3",
    "prompt": promptQuestion + text,
    }
    
    extract_claim_response = requests.post(llama3_url, json=extract_claim_payload)
    
    if extract_claim_response.status_code == 200:
        lines = extract_claim_response.text.splitlines()
        output = ""
        for line in lines:
            try:
                # Parse each line as JSON
                line_data = json.loads(line)
                # Concatenate the "response" field
                output += line_data.get("response", "")
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue
        # Parse the final concatenated output as JSON
        try:
            final_output = json.loads(output)
            print("Extracted Claims:", final_output)
            return final_output
        except json.JSONDecodeError:
            # Add a } to the end of the output
            output += "}"
            print("Output after adding }:", output)
            try:
                final_output = json.loads(output)
                print("Extracted Claims:", final_output)
                return final_output
            except json.JSONDecodeError:
                print("Final output is not valid JSON:", output)
                return None
    else:
        print(f"Error: {extract_claim_response.status_code}")
        print(extract_claim_response.text)

In [18]:
ids_to_claims = {}
for index, row in tqdm.tqdm(pandas_df.iterrows(), total=pandas_df.shape[0]):
    # Extract the text from the DataFrame
    text = row['ad_creative_bodies']
    id = row['ad_archive_id']
    # Extract claims from the text
    claims = extract_claims(text)
    # Store the claims in the dictionary
    ids_to_claims[id] = claims
    
# Check if the folder and file exist
if not os.path.exists("./PreprocessedDataset/"):
    os.makedirs("./PreprocessedDataset/")
    

# Save the claims to a CSV file
with open("./PreprocessedDataset/" + party + "_claims.csv", "w", newline='', encoding='utf-8') as csvfile:
    fieldnames = ['ad_archive_id', 'claims']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for id, claims in ids_to_claims.items():
        writer.writerow({'ad_archive_id': id, 'claims': json.dumps(claims)})

  4%|▎         | 1/27 [00:22<09:45, 22.52s/it]

Extracted Claims: {'Claims': ['The anti-Albanian quarters have a tendency to abolish amnesty and distort the fight for freedom and equality of UÃ‡K.', 'Gordana Siljanovska has proven the same thing from her own mouth today.', "It is necessary to prove that VMRO-DPMNE's political battle is to subjugate Albanians, not to continue the partnership established in 2001!", "The European Front should win over 22 mandates and 'Siljanovskas' should remain only a memory of posters and billboards."]}


  7%|▋         | 2/27 [00:27<04:57, 11.91s/it]

Extracted Claims: {'Claims': ['Macedonia is yours again!', "It's time for a change!"]}


 11%|█         | 3/27 [00:29<02:59,  7.49s/it]

...

Output after adding }: {
"Claims": [
"The candidate has been a member of VMRO-DPMNE for over 20 years",
"He is a representative of the IJ movement and has been actively involved in its activities",
"He has worked with several organizations and institutions, including the Macedonian Orthodox Church",
"The candidate has been recognized for his work and dedication to the Macedonian people",
"He has a strong commitment to the country's independence and sovereignty"
]}
Extracted Claims: {'Claims': ['The candidate has been a member of VMRO-DPMNE for over 20 years', 'He is a representative of the IJ movement and has been actively involved in its activities', 'He has worked with several organizations and institutions, including the Macedonian Orthodox Church', 'The candidate has been recognized for his work and dedication to the Macedonian people', "He has a strong commitment to the country's independence and sovereignty"]}


 96%|█████████▋| 26/27 [07:03<00:20, 20.17s/it]

Output after adding }: {
"Claims": [
"A survey by IPIS shows a two-way advantage for the opposition party VMRO-DPMNE in Macedonia compared to LSDM.",
"A survey published earlier on social networks, attributed to IPIS, was confirmed as falsified by the director of the institute, Vladimir Bozhinovski.",
"Analysts emphasize the importance of opinion polls at the eve of elections, but note that financing from political parties can affect their objectivity."
]}
Extracted Claims: {'Claims': ['A survey by IPIS shows a two-way advantage for the opposition party VMRO-DPMNE in Macedonia compared to LSDM.', 'A survey published earlier on social networks, attributed to IPIS, was confirmed as falsified by the director of the institute, Vladimir Bozhinovski.', 'Analysts emphasize the importance of opinion polls at the eve of elections, but note that financing from political parties can affect their objectivity.']}


100%|██████████| 27/27 [07:24<00:00, 16.45s/it]

Output after adding }: {
"Claims": [
"President Stevo Pendarovski will run for a second term with support from LSDM.",
"Gordana Siljanovska Davkova will run as the candidate of VMRO-DPMNE.",
"This will be the second matchup between Pendarovski and Siljanovska-DavkovÃ«s, after the 2019 presidential elections.",
"Arben Taravari has been officially promoted as a presidential candidate by VLEN."}
Final output is not valid JSON: {
"Claims": [
"President Stevo Pendarovski will run for a second term with support from LSDM.",
"Gordana Siljanovska Davkova will run as the candidate of VMRO-DPMNE.",
"This will be the second matchup between Pendarovski and Siljanovska-DavkovÃ«s, after the 2019 presidential elections.",
"Arben Taravari has been officially promoted as a presidential candidate by VLEN."}


### Claim to Questions

In [19]:
def claim_to_question(claim):
    prompt_question = f""" For the bellow claim generate me 3 neural fact-checking questions and give to me in the format of a json. You should only return the json. The json should have only 1 key `Questions` and the value should be a list of questions.
    Claim: "{claim}"""
    
    extract_claim_payload = {
    "model": "llama3",
    "prompt": prompt_question + text,
    }
    
    question_response = requests.post(llama3_url, json=extract_claim_payload)
    
    if question_response.status_code == 200:
        # Process the streamed response
        lines = question_response.text.splitlines()
        output = ""

        for line in lines:
            try:
                # Parse each line as JSON
                line_data = json.loads(line)
                # Concatenate the "response" field
                output += line_data.get("response", "")
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue

        # Parse the final concatenated output as JSON
        try:
            final_output = json.loads(output)
            print("Generated Questions:", final_output)
            return final_output
        except json.JSONDecodeError:
            # Add a } to the end of the output
            output += "}"
            try:
                final_output = json.loads(output)
                print("Generated Questions:", final_output)
                return final_output
            except json.JSONDecodeError:
                print("Final output is not valid JSON:", output)
                return None
    else:
        print(f"Error: {question_response.status_code}")
        print(question_response.text)

In [36]:
# Open the CSV file and read the claims
with open("./PreprocessedDataset/" + party + "_claims.csv", "r", encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    claims_data = list(reader)

pd_claims_data = pd.DataFrame(claims_data)

for index, row in tqdm.tqdm(pd_claims_data.iterrows(), total=pd_claims_data.shape[0]):
    # Extract the claims from the DataFrame
    claims = json.loads(row['claims'])
    id = row['ad_archive_id']
    print("ID", id)
    print("Claims", claims)

100%|██████████| 27/27 [00:00<00:00, 3123.69it/s]

ID 427480589881211
Claims {'Claims': ['The anti-Albanian quarters have a tendency to abolish amnesty and distort the fight for freedom and equality of UÃ‡K.', 'Gordana Siljanovska has proven the same thing from her own mouth today.', "It is necessary to prove that VMRO-DPMNE's political battle is to subjugate Albanians, not to continue the partnership established in 2001!", "The European Front should win over 22 mandates and 'Siljanovskas' should remain only a memory of posters and billboards."]}
ID 1636678057070325
Claims {'Claims': ['Macedonia is yours again!', "It's time for a change!"]}
ID 252100357930225
Claims {'Claims': ['Many years ago']}
ID 1117244869521215
Claims {'Claims': ['Gordana Siljanovska-Davkov won the first round of presidential elections in North Macedonia held on April 24.', 'She secured a decisive victory with 363,086 votes (40.08%) and gained 182,586 more votes than Stevo Pendarovski, the current president who received 180,500 votes (19.92%).', 'Bujar Osmani from

In [37]:
# Load the claims from the CSV file
claims_df = pd.read_csv("./PreprocessedDataset/" + party + "_claims.csv")
print(claims_df.head())

ids_to_claims_to_questions = {}

for index, row in tqdm.tqdm(claims_df.iterrows(), total=claims_df.shape[0]):
    claims_to_questions = {}
    # Extract the claims from the DataFrame
    claims = json.loads(row['claims'])
    id = row['ad_archive_id']
    # Extract claims from the text
    for claim in tqdm.tqdm(claims['Claims']):
        questions = claim_to_question(claim)
        # Store the claims in the dictionary
        claims_to_questions[claim] = questions
    # Store the claims in the dictionary
    ids_to_claims_to_questions[id] = claims_to_questions
# Check if the folder and file exist
if not os.path.exists("./PreprocessedDataset/"):
    os.makedirs("./PreprocessedDataset/")
# Save the claims to a CSV file
with open("./PreprocessedDataset/" + party + "_claims_to_questions.csv", "w", newline='', encoding='utf-8') as csvfile:
    fieldnames = ['ad_archive_id', 'claims_to_questions']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for id, claims in ids_to_claims_to_questions.items():
        writer.writerow({'ad_archive_id': id, 'claims_to_questions': json.dumps(claims)})
        

      ad_archive_id                                             claims
0   427480589881211  {"Claims": ["The anti-Albanian quarters have a...
1  1636678057070325  {"Claims": ["Macedonia is yours again!", "It's...
2   252100357930225                     {"Claims": ["Many years ago"]}
3  1117244869521215  { "Claims": [ "Gordana Siljanovska-Davkov won ...
4  1595858924569626  {"Claims": ["Your trust in politicians is lost...


  0%|          | 0/27 [00:00<?, ?it/s]

Generated Questions: {'Questions': ["Is 'anti-Albanian quarters' a neutral or objective term?", 'What evidence exists to support the claim that these anti-Albanian quarters tend to abolish amnesty?', 'Do historical records show that UÃ‡K has been fighting for freedom and equality, rather than distorting it?']}


Generated Questions: {'Questions': ['What does Gordana Siljanovska claim to have proven?', 'Who is she claiming has been determined for change?', "What are her predictions regarding VLEN's chances and the candidacy of Arben Taravari?"]}


Generated Questions: {'Questions': [["Is there evidence that VMRO-DPMNE's political battle is primarily focused on subjugating Albanians?"], ['Does the 2001 partnership established by VMRO-DPMNE include provisions for ensuring Albanian rights and interests?'], ['Are there credible sources indicating that the people are seeking change, and if so, what specific aspects of governance or policy are they looking to alter?']]}


...

Generated Questions: {'Questions': ['What specific investment figure does the claim mention for the development of high-voltage electrical networks?', 'What type of infrastructure projects does the claim mention being improved across Macedonia?', "Who are mentioned as individuals related to the claim's context (if any)?"]}


 41%|████      | 11/27 [10:17<15:03, 56.44s/it]

Generated Questions: {'Questions': ["How does the claim compare to VMRO-DPMNE's promises during their 11-year term?", 'What evidence supports the claim that VLEN has already proven its ability to increase support?', 'What is the basis for the assertion that Arben Taravari will be the winning candidate?']}


 41%|████      | 11/27 [10:24<15:09, 56.81s/it]


KeyboardInterrupt: 

## Google Search (Evidence) Retrieval

In [91]:
def google_search(query, api_key, cse_id, num=3):
    query = quote_plus(query)
    url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={api_key}&cx={cse_id}&num={num}"
    response = requests.get(url).json()
    
    results = []
    for item in response.get('items', []):
        results.append({
            'title': item.get('title'),
            'link': item.get('link'),
            'snippet': item.get('snippet')
        })
    return results[:5] 

In [94]:
Evidence = {}
Claims_and_Evidence_dict = {}
for claim, questions in claim_to_question_dic.items():
    print(claim)
    print(questions)
    for question in questions:
        search_results = google_search(question, google_api, cx)
        Evidence[question] = search_results
    Claims_and_Evidence_dict[claim] = Evidence
    Evidence = {}
    
    
    
    # for question in questions:
    #     search_results = google_search(question, google_api, cx)
    #     Evidence[question] = search_results
    # Claims_and_Evidence_dict[claim] = Evidence
    # Evidence = {}
    # print(Claims_and_Evidence_dict)

Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren
["What is the purpose of singing national anthems according to Hristijan Mickoski's proposal?", 'Is it true that VMRO-DPMNE promotes nationalism and anti-European agendas, as claimed by the author?', 'Does the author suggest that VMRO-DPMNE isolates the country and hinders its future prospects in Europe?']
The proposal of the leader of VMRO-DPMNE Hristijan Mickoski to make children sing the national anthem every month reflects the nationalist mindset of VMRO-DPMNE
['Is the proposal to make children sing the national anthem every month a common practice in any country?', 'What is the current state of education in North Macedonia regarding national anthems, and is it true that VMRO-DPMNE is trying to impose its nationalist mindset on children?', "Does Hristijan Mickoski's proposal reflect a broader nationalistic agenda by VMRO-DPMNE, and if so, what are the implications for the country's future integration into Euro

In [ ]:
# Print Claims_and_Evidence_dict in a pretty format
print(json.dumps(Claims_and_Evidence_dict, indent=4))

{
    "Himni dhe simbolet shtet\u00ebrore kan\u00eb vendin dhe koh\u00ebn e vet se ku p\u00ebrdoren": {
        "What is the purpose of singing national anthems according to Hristijan Mickoski's proposal?": [
            {
                "title": "North Macedonia's parliament approves a new center-right ...",
                "link": "https://apnews.com/article/north-macedonia-mickoski-government-8021f259cd9b8fd3f128f35f533ab481",
                "snippet": "Jun 23, 2024 ... ... proposed by a center-right party that won May's national elections. ... Hristijan Mickoski, 46, faces significant challenges in his four\u00a0..."
            },
            {
                "title": "MONOETHNIC ANTHEM: Education under the claws of nationalism ...",
                "link": "https://koha.mk/en/himni-monoetnik-arsimi-nen-kthetrat-e-nacionalizmit/",
                "snippet": "Dec 16, 2024 ... The law on secondary education that foresees the singing of the national anthem in schools is the result

## Identify Stance of Evidence

In [96]:
def detect_stance(claim, evidence):
    prompt = f"""Analyze whether this evidence supports or refutes the claim.
    Claim: "{claim}"
    Evidence: "{evidence}"
    Respond only with on of the following options: Completely Supports, Partially Support, Refute, Irrelevant, or No Evidence.
    """
    extract_stance_payload = {
    "model": "llama3",
    "prompt": prompt,
    }
    stance_response = requests.post(llama3_url, json=extract_stance_payload)
    if stance_response.status_code == 200:
        # Process the streamed response
        lines = stance_response.text.splitlines()
        output = ""

        for line in lines:
            try:
                # Parse each line as JSON
                line_data = json.loads(line)
                # Concatenate the "response" field
                output += line_data.get("response", "")
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue

        # Parse the final concatenated output as JSON
        # Remove everything after the newline character
        output = output.split('\n')[0]
        return output
    else:
        print(f"Error: {stance_response.status_code}")
        print(stance_response.text)
    return None

In [105]:
stance_results = {}


for claim, evidence in tqdm.tqdm(Claims_and_Evidence_dict.items()):
    for question, evidences in tqdm.tqdm(evidence.items()):
        for evidence in tqdm.tqdm(evidences):
            stance = detect_stance(claim, evidence)
            if claim not in stance_results:
                stance_results[claim] = {}
            if question not in stance_results[claim]:
                stance_results[claim][question] = []
            stance_results[claim][question].append({
                'evidence': evidence,
                'stance': stance
            })

100%|██████████| 7/7 [07:07<00:00, 61.02s/it]


In [106]:
# Print stance_results in a pretty format
print(json.dumps(stance_results, indent=4))

{
    "Himni dhe simbolet shtet\u00ebrore kan\u00eb vendin dhe koh\u00ebn e vet se ku p\u00ebrdoren": {
        "What is the purpose of singing national anthems according to Hristijan Mickoski's proposal?": [
            {
                "evidence": {
                    "title": "North Macedonia's parliament approves a new center-right ...",
                    "link": "https://apnews.com/article/north-macedonia-mickoski-government-8021f259cd9b8fd3f128f35f533ab481",
                    "snippet": "Jun 23, 2024 ... ... proposed by a center-right party that won May's national elections. ... Hristijan Mickoski, 46, faces significant challenges in his four\u00a0..."
                },
                "stance": "Partially Supports"
            },
            {
                "evidence": {
                    "title": "MONOETHNIC ANTHEM: Education under the claws of nationalism ...",
                    "link": "https://koha.mk/en/himni-monoetnik-arsimi-nen-kthetrat-e-nacionalizmit/",
   

## Determine Correction

In [ ]:
def determine_verdict(stance_results):
    verdicts = []
    for claim, questions in stance_results.items():
        for question, evidences in questions.items():
            # Make sure that all the stance results are in the same format
            for evidence in evidences:
                for key in evidence:
                    if key == 'stance':
                        stance = evidence['stance']
                        
                        if stance != 'Completely Supports' and stance != 'Partially Support' and stance != 'Refute':
                            evidence['stance'] = 'Irrelevant'
                            # Search if the sentence contains any of the words
                            if 'Completely Supports' in stance:
                                evidence['stance'] = 'Completely Supports'
                            elif 'Partially Support' in stance:
                                evidence['stance'] = 'Partially Support'
                            elif 'Refute' in stance:
                                evidence['stance'] = 'Refute'
                            elif 'Irrelevant' in stance:
                                evidence['stance'] = 'Irrelevant'
                            elif 'No Evidence' in stance:
                                evidence['stance'] = 'No Evidence'
                        print(stance)
                    
            
            supports = sum(1 for evidence in evidences if evidence['stance'] == 'Completely Supports' or evidence['stance'] == 'Partially Support')
            refutes = sum(1 for evidence in evidences if evidence['stance'] == 'Refute')
            if supports > refutes:
                verdicts.append((claim, question, "Supports"))
            elif refutes > supports:
                verdicts.append((claim, question, "Refutes"))
            else:
                verdicts.append((claim, question, "Neutral"))
                
            
            
    return verdicts

In [124]:
determine_verdict(stance_results)

Partially Support
Partially Support
Refute
Partially Support
Partially Support
Partially Support
Partially Support
No Evidence
Irrelevant
Refute
Refute
Refute
Refute
Refute
Refute
Refute
Refute
Partially Support
Partially Support
Partially Support
No Evidence
Partially Support
Partially Support
Partially Support
Partially Support
Irrelevant
Refute
No Evidence
Partially Support
Partially Support
Irrelevant
Irrelevant
Irrelevant
Partially Support
Partially Support
Partially Support
Irrelevant
Partially Support
Partially Support
Partially Support
Partially Support
Partially Support
No Evidence
Partially Support
Partially Support
No Evidence
Partially Support
No Evidence
No Evidence
Partially Support
Partially Support
No Evidence
Partially Support
Partially Support
No Evidence
Irrelevant
Irrelevant


[('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  "What is the purpose of singing national anthems according to Hristijan Mickoski's proposal?",
  'Supports'),
 ('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  'Is it true that VMRO-DPMNE promotes nationalism and anti-European agendas, as claimed by the author?',
  'Supports'),
 ('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  'Does the author suggest that VMRO-DPMNE isolates the country and hinders its future prospects in Europe?',
  'Supports'),
 ('The proposal of the leader of VMRO-DPMNE Hristijan Mickoski to make children sing the national anthem every month reflects the nationalist mindset of VMRO-DPMNE',
  'Is the proposal to make children sing the national anthem every month a common practice in any country?',
  'Refutes'),
 ('The proposal of the leader of VMRO-DPMNE Hristijan Mickoski to make children sing the national anthem every month re

In [125]:
def claims_verdict(claims, stance_results):
    verdicts = []
    for claim in claims:
        supports = sum(1 for question, evidences in stance_results[claim].items() if any(evidence['stance'] == 'Completely Supports' or evidence['stance'] == 'Partially Support' for evidence in evidences))
        refutes = sum(1 for question, evidences in stance_results[claim].items() if any(evidence['stance'] == 'Refute' for evidence in evidences))
        if supports > refutes:
            verdicts.append((claim, "Supports"))
        elif refutes > supports:
            verdicts.append((claim, "Refutes"))
        else:
            verdicts.append((claim, "Neutral"))
    return verdicts
claims_verdict(claims, stance_results)

[('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  'Supports'),
 ('The proposal of the leader of VMRO-DPMNE Hristijan Mickoski to make children sing the national anthem every month reflects the nationalist mindset of VMRO-DPMNE',
  'Refutes'),
 ('VMRO-DPMNE spreads unjust nationalism while making excuses for anti-European agendas',
  'Supports'),
 ('VMRO-DPMNE fosters hatred and division', 'Supports'),
 ('VMRO-DPMNE blocks the European future of young generations', 'Supports'),
 ('VMRO-DPMNE isolates the country', 'Supports'),
 ('VMRO-DPMNE exercises power against Albanian nation in the municipalities that lead its quotas',
  'Supports')]